In [1]:
import taichi as ti
from taichi.math import exp, cos, sin, pi, sqrt
ti.reset()
ti.init(arch = ti.vulkan, fast_math=True)

[Taichi] version 1.7.4, llvm 15.0.7, commit b4b956fd, osx, python 3.10.19


[I 11/09/25 01:49:32.559 5023240] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


[Taichi] Starting on arch=vulkan


RHI Error: Potential non-conformant Vulkan implementation, enabling VK_KHR_portability_subset


In [ ]:
# Number of pixels in our grid
n       = 300
dx      = 1/n
dt      = 1e-1 * (2*dx*dx)
wave    = ti.Vector.field(2, ti.f32, (n,n))
wavenew = ti.Vector.field(2, ti.f32, (n,n))
pixels  = ti.Vector.field(3, ti.f32, (n,n))
V       = ti.field(ti.f32, (n,n))
gui     = ti.GUI("2D Waves", res = n, fast_gui=True)

In [ ]:
@ti.kernel
def fill_V(V:ti.template(),n:int):
    for x,y in V:
        if (2*x-n)**2 + (2*y-n)**2 <(0.2*n)**2:
            V[x,y] = 1.e5

# @ti.func
# def v(x,y):
#     A = 0.
#     if (2*x-n)**2 + (2*y-n)**2 <(0.2*n)**2:
#         A = 1.e5

#     return A

@ti.kernel
def initialize(x:float, y:float, px:float, py:float, sx:float, sy:float):
    for i,j in wave:
        if i!=0 and j!=0 and i!=n-1 and j!=n-1: 
            psi = exp(-((2*i-n)/n - x)**2/(2*sx**2) - ((2*j-n)/n - y)**2/(2*sy**2))#/(2*pi*sx*sy)
            wave[i,j][0] = psi * cos((2*i-n)/n*px + (2*j-n)/n*py)
            wave[i,j][1] = psi * sin((2*i-n)/n*px + (2*j-n)/n*py)
        else:
            wave[i,j] = 0
    
    h = dt/(4*dx*dx)
    for i,j in wave:
        if i!=0 and j!=0 and i!=n-1 and j!=n-1: 
            wave[i,j][1] = wave[i,j][1] + (- 4*h - dt*V[i,j]/2)*wave[i,j][0] + h*(wave[i+1,j][0] + wave[i-1,j][0] + wave[i,j+1][0] + wave[i,j-1][0])

# @ti.func
# def step_real(i:int, j:int, dx:float, dt:float, n:int):
#     h = dt/(2*dx*dx)
#     wavenew[i,j][0] = wave[i,j][0] - (- 4*h - dt*V[i,j])*wave[i,j][1] - h*(wave[i+1,j][1] + wave[i-1,j][1] + wave[i,j+1][1] + wave[i,j-1][1])
    
# @ti.func
# def step_imag(i:int, j:int, dx:float, dt:float, n:int):
#     h = dt/(2*dx*dx)
#     wavenew[i,j][1] = wave[i,j][1] + (- 4*h - dt*V[i,j])*wavenew[i,j][0] + h*(wavenew[i+1,j][0] + wavenew[i-1,j][0] + wavenew[i,j+1][0] + wavenew[i,j-1][0])

@ti.kernel
def draw(n:int, dt:float, dx:float):
    h = dt/(2*dx*dx)
    for i,j in ti.ndrange((1, n-1), (1, n-1)): #wave:
        wavenew[i,j][0] = wave[i,j][0] - (- 4*h - dt*V[i,j])*wave[i,j][1] - h*(wave[i+1,j][1] + wave[i-1,j][1] + wave[i,j+1][1] + wave[i,j-1][1])
        # if i!=0 and j!=0 and i!=n-1 and j!=n-1: 
        #     step_real(i,j,dx,dt,n)
    
    for i,j in ti.ndrange((1, n-1), (1, n-1)):# wave:
        wavenew[i,j][1] = wave[i,j][1] + (- 4*h - dt*V[i,j])*wavenew[i,j][0] + h*(wavenew[i+1,j][0] + wavenew[i-1,j][0] + wavenew[i,j+1][0] + wavenew[i,j-1][0])
        # if i!=0 and j!=0 and i!=n-1 and j!=n-1: 
        #     step_imag(i,j,dx,dt,n)

        
    for i,j in pixels:
        pixels[i,j] = wavenew[i,j][0]**2 + wave[i,j][1]*wavenew[i,j][1]
        wave[i,j]   = wavenew[i,j]

In [ ]:
initialize(0.6,0,-100,0,1e-1,1e-1)
fill_V(V,n)

while gui.running:    
    draw(n,dt,dx)

    gui.set_image(pixels)
    gui.show()